In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
display(HTML("<style>.output_result { max-width:95% !important; }</style>"))
 
#여백 줄이기
display(HTML("<style>.prompt { min-width: 1ex !important; }</style>"))

#In 부분 없애기
#display(HTML("<style>.prompt bdi { display:none !important; }</style>"))
 
#아예 다 없애기
#display(HTML("<style>.prompt { display:none !important; }</style>"))

/tmp/ipykernel_18314/3328681235.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [17]:
import re

custom_prompt = """Your task is to find the changed domain-slots based on the context and the dialogue between user and system, and find the corresponding value.
The following lists are domain-slots and their possible values.
you don't have to find other changed domain-slots if they are not in the list.

hotel-name: a and b guest house, ashley hotel, el shaddia guest house, etc.
hotel-pricerange: dontcare, cheap, moderate, expensive
hotel-type: hotel, guest house
hotel-parking: dontcare, yes, no
hotel-book_stay: 1, 2, 3, etc.
hotel-book_day: monday, tuesday, etc.
hotel-book_people: 1, 2, 3, etc.
hotel-area: dontcare, centre, east, north, south, west
hotel-stars: dontcare, 0, 1, 2, 3, 4, 5
hotel-internet: dontcare, yes, no

train-destination: london kings cross, cambridge, peterborough, etc.
train-departure: cambridge, stansted airport, etc.
train-day: monday, saturday, etc.
train-book_people: 1, 2, 3, etc.
train-leaveat: 20:24, 12:06, etc.
train-arriveby: 05:51, 20:52, etc.

attraction-name: abbey pool and astroturf pitch, adc theatre, all saints church, castle galleries, etc.
attraction-area: dontcare, centre, east, north, south, west
attraction-type: architecture, boat, church, cinema, college, concert hall, entertainment, hotspot, multiple sports, museum, nightclub, park, special, swimming pool, theatre

restaurant-name: pizza hut city centre, the missing sock, golden wok, cambridge chop house, darrys cookhouse and wine shop, etc.
restaurant-food: italian, international, chinese, dontcare, modern european, etc.
restaurant-pricerange: dontcare, cheap, moderate, expensive
restaurant-area: centre, east, north, south, west
restaurant-book_time: 13:30, 17:11, etc.
restaurant-book_day: wednesday, friday, etc.
restaurant-book_people: 1, 2, 3, etc.

taxi-destination: copper kettle, magdalene college, lovell lodge
taxi-departure: royal spice, university arms hotel, da vinci pizzeria
taxi-leaveat: 14:45, 11:15, etc.
taxi-arriveby: 15:30, 12:45, etc.
"""

slot_classify_prompt = """Your task is to find the changed domain-slots based on the context and the dialogue between user and system, and find the corresponding value.
The following lists are domain-slots and their possible values.
All domain-slots have their own number that represent themselves in examples.

domain-slots (number): possible values
hotel-name (0): a and b guest house, ashley hotel, el shaddia guest house, etc.
hotel-pricerange (1): dontcare, cheap, moderate, expensive
hotel-type (2): hotel, guest house
hotel-parking (3): dontcare, yes, no
hotel-book stay (4): 1, 2, 3, etc.
hotel-book day (5): monday, tuesday, etc.
hotel-book people (6): 1, 2, 3, etc.
hotel-area (7): dontcare, centre, east, north, south, west
hotel-stars (8): dontcare, 0, 1, 2, 3, 4, 5
hotel-internet (9): dontcare, yes, no
train-destination (10): london kings cross, cambridge, peterborough, etc.
train-departure (11): cambridge, stansted airport, etc.
train-day (12): monday, saturday, etc.
train-book people (13): 1, 2, 3, etc.
train-leaveat (14): 20:24, 12:06, etc.
train-arriveby (15): 05:51, 20:52, etc.
attraction-name (16): abbey pool and astroturf pitch, adc theatre, all saints church, castle galleries, etc.
attraction-area (17): dontcare, centre, east, north, south, west
attraction-type (18): architecture, boat, church, cinema, college, concert hall, entertainment, hotspot, multiple sports, museum, nightclub, park, special, swimming pool, theatre
restaurant-name (19): pizza hut city centre, the missing sock, golden wok, cambridge chop house, darrys cookhouse and wine shop, etc.
restaurant-food (20): italian, international, chinese, dontcare, modern european, etc.
restaurant-pricerange (21): dontcare, cheap, moderate, expensive
restaurant-area (22): centre, east, north, south, west
restaurant-book time (23): 13:30, 17:11, etc.
restaurant-book day (24): wednesday, friday, etc.
restaurant-book people (25): 1, 2, 3, etc.
taxi-destination (26): copper kettle, magdalene college, lovell lodge
taxi-departure (27): royal spice, university arms hotel, da vinci pizzeria
taxi-leaveat (28): 14:45, 11:15, etc.
taxi-arriveby (29): 15:30, 12:45, etc.
"""

def conversion(prompt, reverse=False):
    conversion_dict = {"leaveat": "depart_time", "arriveby": "arrive_by_time",
                       "book_stay": "book_number_of_days",
                       "food": "food_type"}
    reverse_conversion_dict = {v: k for k, v in conversion_dict.items()}
    used_dict = reverse_conversion_dict if reverse else conversion_dict

    for k, v in used_dict.items():
        prompt = prompt.replace(k, v)
    return prompt

def pred_parse_with_bracket_match(pred):

    # find all values where they are in the brackets

    # fix for no states
    if pred == "":
        return {}

    pred_slot_values = {}

    # slot_value = pred.split(",")
    slot_value = re.findall(r'\((.*?)\)', pred)

    value_assigner = "="
    for i in slot_value:
      i = i.replace("(","").replace(")","")
      if value_assigner not in i:
        continue
      else:
        pred_slot_values[i.split(value_assigner)[0].strip()] = i.split(value_assigner)[1].strip()

    return pred_slot_values

def slot_classify_parse(pred):

    # # the output format is "(slot_name = value)"
    # start_pos = pred.rfind("(") + 1
    # end_pos = pred.rfind(")")
    # pred = pred[start_pos:end_pos]

    # fix for no states
    if pred == "":
        return {}

    pred_slot_values = {}

    slot_value = pred.split(",")

    value_assigner = "="
    for i in slot_value:
      i = i.replace("(","").replace(")","")
      if value_assigner not in i:
        continue
      else:
        print(i)
        pred_slot = idx_to_slot(int(i.split(value_assigner)[0].strip()))
        pred_value = i.split(value_assigner)[1].strip()
        print(pred_slot + " " + pred_value)
        pred_slot_values[pred_slot] = pred_value

    return pred_slot_values

slot_idx_dict = {'hotel-name': 0,
 'hotel-pricerange': 1,
 'hotel-type': 2,
 'hotel-parking': 3,
 'hotel-book stay': 4,
 'hotel-book day': 5,
 'hotel-book people': 6,
 'hotel-area': 7,
 'hotel-stars': 8,
 'hotel-internet': 9,
 'train-destination': 10,
 'train-departure': 11,
 'train-day': 12,
 'train-book people': 13,
 'train-leaveat': 14,
 'train-arriveby': 15,
 'attraction-name': 16,
 'attraction-area': 17,
 'attraction-type': 18,
 'restaurant-name': 19,
 'restaurant-food': 20,
 'restaurant-pricerange': 21,
 'restaurant-area': 22,
 'restaurant-book time': 23,
 'restaurant-book day': 24,
 'restaurant-book people': 25,
 'taxi-destination': 26,
 'taxi-departure': 27,
 'taxi-leaveat': 28,
 'taxi-arriveby': 29}

def slot_to_idx(str):
    # str = str.replace(" ","_")
    if str in slot_idx_dict.keys():
        return slot_idx_dict[str]
    else:
        return -1

def idx_to_slot(idx):
    if idx in slot_idx_dict.values():
        return {v: k for k, v in slot_idx_dict.items()}[idx]
    else:
        return "invalid_slot_idx"
    
aaa = "(restaurant-name = nusha), and the implicit domain-slot is (restaurant-food = dontcare)."
aaa = "(restaurant-pricerange = expensive), (restaurant-food = indian)."
aaa = "(taxi-leaveat = 17:15), ()."
aaa = "N/A."
aaa = "(attraction-name = nusha), (restaurant-name = nandos)."

pred_parse_with_bracket_match(aaa)

{'attraction-name': 'nusha', 'restaurant-name': 'nandos'}

# Answer-aware CoT example

# GPT3.5 turbo Baseline

In [45]:
import json

running_log_path = "expts/gpt35_turbo_5p_v2_ours/running_log.json"
with open(running_log_path,'r') as f:
    content = json.load(f)
    
for idx, i in enumerate(content):
    completion = i["completion"].replace(traditional_prompt, "")
    pred = our_pred_parse(completion)
    print(idx)
    print("sys  uttr  ==== " + i["dialog"]["sys"][-1])
    print("user uttr  ==== " + i["dialog"]["usr"][-1])
    print()
    print("prev_label ==== " + str(i["last_slot_values"]))
    print()
    print("completion ==== " + i["completion"].replace(traditional_prompt, ""))
    print()
    print("prediction ==== " + str(pred))
    print("labeling.. ==== " + str(i["turn_slot_values"]))
    print("\n\n\n\n\n\n")

0
sys  uttr  ==== 
user uttr  ==== i would like a taxi from saint john s college to pizza hut fen ditton .

prev_label ==== {}

completion ==== taxi-departure = saint john s college, taxi-destination = pizza hut fen ditton

prediction ==== {'taxi-departure': 'saint john s college', 'taxi-destination': 'pizza hut fen ditton'}
labeling.. ==== {'taxi-destination': 'pizza hut fenditton', 'taxi-departure': 'saint johns college'}







1
sys  uttr  ==== what time do you want to leave and what time do you want to arrive by ?
user uttr  ==== i want to leave after 17:15 .

prev_label ==== {'taxi-destination': 'pizza hut fenditton', 'taxi-departure': 'saint johns college'}

completion ==== taxi-leaveat = 17:15

prediction ==== {'taxi-leaveat': '17:15'}
labeling.. ==== {'taxi-leaveat': '17:15'}







2
sys  uttr  ==== booking completed ! your taxi will be blue honda contact number is 07218068540
user uttr  ==== thank you for all the help ! i appreciate it .

prev_label ==== {'taxi-leaveat': '17

In [ ]:
6, 9, 16, 18, 19, 26, 32, 33

# Slot label provided

In [26]:
running_log_path = "expts/230416_0330-gpt35_turbo_5p_v2_ZeroCoT_slotlabel_provide/running_log.json"
with open(running_log_path,'r') as f:
    content = json.load(f)
            
# for idx, i in enumerate(content):
#     if i["pred_status"] == "wrong":
#         completion = i["completion"].replace(traditional_prompt, "")
#         pred = our_pred_cot_parse(completion)
#         if i["predicted_slot_values"] != i["turn_slot_values"]:
#             print(idx)
# #             print(i["prompt"].replace(traditional_prompt, ""))
#             print("sys  uttr  ==== " + i["dialog"]["sys"][-1])
#             print("user uttr  ==== " + i["dialog"]["usr"][-1])
#             print()
#             print("prev_label ==== " + str(i["last_slot_values"]))
#             print()
#             print("completion ==== " + i["completion"].replace(traditional_prompt, ""))
#             print()
#             print("prediction ==== " + str(i["predicted_slot_values"]))
#             print("labeling.. ==== " + str(i["turn_slot_values"]))
#             print("\n\n\n\n\n\n")

for idx, i in enumerate(content):
        completion = i["completion"].replace(traditional_prompt, "")
        pred = our_pred_cot_parse(completion)
        print(idx)
        print("sys  uttr  ==== " + i["dialog"]["sys"][-1])
        print("user uttr  ==== " + i["dialog"]["usr"][-1])
        print()
        print("prev_label ==== " + str(i["last_slot_values"]))
        print()
        print("completion ==== " + i["completion"].replace(traditional_prompt, ""))
        print()
        print("prediction ==== " + str(i["predicted_slot_values"]))
        print("labeling.. ==== " + str(i["turn_slot_values"]))
        print("\n\n\n\n\n\n")

0
sys  uttr  ==== 
user uttr  ==== i would like a taxi from saint john s college to pizza hut fen ditton .

prev_label ==== {}

completion ==== 1. User expresses their desire to book a taxi.
2. User provides the departure location as "saint john s college" and the destination as "pizza hut fen ditton".
3. System confirms the information and asks for the departure time.
4. User provides the departure time.
5. System confirms the information and asks for the arrival time.
6. User does not provide an arrival time, so the system asks again.
7. User provides the arrival time.
8. System confirms the information and provides the estimated fare.
9. User confirms the booking and ends the conversation.

Based on the above dialogue, we can see that the slots taxi-departure and taxi-destination were changed and their values were set to saint john s college and pizza hut fen ditton respectively. Therefore, the changed slot and value (slot_name = value) is (taxi-departure = saint john s college, tax

# slot classify with slot desc 0t0368

In [7]:
import json

running_log_path = "expts/230419_1637-gpt35_turbo_5p_v2_slot_classify_with_desc_0to368/running_log.json"
with open(running_log_path,'r') as f:
    content = json.load(f)

for idx, i in enumerate(content):
    if i["pred_status"] == "wrong":
        completion = i["completion"].replace(traditional_prompt, "")
        if i["predicted_slot_values"] != i["turn_slot_values"]:
            print(idx)
            
for idx, i in enumerate(content):
    if i["pred_status"] == "wrong":
        completion = i["completion"].replace(traditional_prompt, "")
        if i["predicted_slot_values"] != i["turn_slot_values"]:
            print(idx)
#             print(i["prompt"].replace(traditional_prompt, ""))
            print("sys  uttr  ==== " + i["dialog"]["sys"][-1])
            print("user uttr  ==== " + i["dialog"]["usr"][-1])
            print()
            print("prev_label ==== " + str(i["last_slot_values"]))
            print()
            print("completion ==== " + i["completion"])
            print()
            print("prediction ==== " + str(i["predicted_slot_values"]))
            print("labeling.. ==== " + str(i["turn_slot_values"]))
            print("\n\n\n\n\n\n") 

6
9
15
16
18
32
33
38
46
58
59
62
63
72
81
86
87
95
98
102
104
110
115
119
122
123
125
126
127
128
135
137
142
143
144
145
148
153
154
155
158
165
166
173
180
183
186
187
191
193
195
200
207
211
213
221
222
226
229
241
242
247
261
264
265
266
271
278
279
280
281
282
289
295
303
304
312
314
317
329
333
336
339
342
344
348
349
359
6
sys  uttr  ==== could you double check that you've spelled the name correctly ? the closest i can find is nandos .
user uttr  ==== it s not a restaurant , it s an attraction . nusha .

prev_label ==== {'restaurant-name': 'nusha'}

completion ==== 16 = nusha

prediction ==== {'attraction-name': 'nusha'}
labeling.. ==== {'attraction-name': 'nusha', 'restaurant-name': '[DELETE]'}







9
sys  uttr  ==== there are 9 indian restaurant -s in centre what price range do you want ?
user uttr  ==== i am looking for expensive indian food .

prev_label ==== {'restaurant-food': 'indian', 'restaurant-area': 'centre', 'attraction-name': 'nusha'}

completion ==== 20 = india

# follow_up_CoT

In [23]:
import json

running_log_path = "expts/230421_1404-gpt35_turbo_5p_v2_follow_up_CoT_0to50/running_log_follow_up_cot.json"
with open(running_log_path,'r') as f:
    content = json.load(f)

for idx, i in enumerate(content):
    if i["pred_status"] == "wrong":
        if i["predicted_slot_values"] != i["turn_slot_values"]:
            print(idx)
            
for idx, i in enumerate(content):
    if i["pred_status"] == "wrong":
        if i["predicted_slot_values"] != i["turn_slot_values"]:
            print(idx)
            print("cot_prompt ==== " + i["follow_up_cot_prompt"].replace(conversion(custom_prompt),""))
            print()
            print("completion ==== " + i["completion"])
            print()
            print("pred change ==== " + str(i["predicted_slot_values"]))
            print("gold change ==== " + str(i["turn_slot_values"]))
            print() 
            print("pred state ==== " + str(i["pred"]))
            print("gold state ==== " + str(i["slot_values"]))
            print("\n\n\n\n\n\n")

6
9
10
15
16
18
19
20
21
22
23
29
31
32
33
34
37
38
39
40
41
42
45
47
48
49
6
cot_prompt ==== 
Example
[context] (restaurant-name = nusha)
[system] could you double check that you've spelled the name correctly ? the closest i can find is nandos .
[user] it s not a restaurant , it s an attraction . nusha .
Q: Based on current dialogue states ([context]), system utterance ([system]), and user utterance ([user]), what domain-slots have been changed and what are their values?
A: the changed domain-slots and values is (attraction-name = nusha), (). Besides, The intent of the user is to correct the system's misunderstanding and provide the correct name of an attraction called "nusha". Also, Yes, the implicit information is that the system initially assumed that the user was referring to a restaurant, but the user corrected the system by stating that it is an attraction. Consequently, the changed domain-slots and values is

completion ==== (attraction-name = nusha).

pred change ==== {'attrac

In [5]:
import os
import json
import math

def loadJson(data_file):
    if os.path.isfile(data_file):
        with open(data_file, 'r') as read_file:
            data = json.load(read_file)
            return data
    
def getBeliefSet(ds):
    bs = set()

    # error handling for the cases that model did not follow the output format.
    for key, value in ds.items():
        t = key+"-"+value
        bs.add(t)
    
#     new_ds = {}
#     for key, value in ds.items():
# #         domain = key.split('-')[0]
# #         slot = key.split('-')[1]
#         domain, slot = key.split('-')
#         if domain not in new_ds:
#             new_ds[domain] = {}
#         new_ds[domain][slot] = value
      
#     for dom in new_ds:
#         for slot in new_ds[dom]:
#             t = dom+"-"+slot+"-"+new_ds[dom][slot]
#             bs.add(t)
            
    return bs

# Flexible Goal Accuracy
def getFGA(gt_list, pr_list, turn_diff, L):
    gt = gt_list[-1]
    pr = pr_list[-1]
    diff1 = gt.symmetric_difference(pr)
    if len(diff1)==0: #Exact match
        return 1
    else:
        if len(gt_list)==1: 
            #Type 1 error
            #First turn is wrong
            return 0
        else:
            diff2 = gt_list[-2].symmetric_difference(pr_list[-2])
            if len(diff2)==0: 
                #Type 1 error
                #Last turn was correct i.e the error in current turn
                return 0
            else:
                tgt = gt.difference(gt_list[-2])
                tpr = pr.difference(pr_list[-2])
                if(not tgt.issubset(pr) or not tpr.issubset(gt)): 
                    #Type 1 error
                    #There exists some undetected/false positive intent in the current prediction
                    return 0
                else:
                    #Type 2 error
                    #Current turn is correct but source of the error is some previous turn
                    return (1-math.exp(-L*turn_diff))


# Run
def FGA(dir_path):
    dst_res_path = os.path.join(dir_path, 'running_log.json')
    dst_res = loadJson(dst_res_path)
    
    fga = [0, 0, 0, 0]
    turn_acc = 0
    total = 0
    lst_lambda = [0.25, 0.5, 0.75, 1.0]

    res = []
    for t in dst_res:
      if t not in res:
        res.append(t)

    for turn in res:
        if turn['turn_id'] == 0:
            gt_list = []
            pr_list = []
            error_turn = -1

        total+=1
        
#         print(turn['pred'])
        gt = getBeliefSet(turn['slot_values'])
        pr = getBeliefSet(turn['pred'])
        gt_list.append(gt)
        pr_list.append(pr)

        m = 0
        for l in range(len(lst_lambda)):
            m = getFGA(gt_list, pr_list, turn['turn_id']-error_turn, lst_lambda[l])
            fga[l]+=m
        if(m==0):
            error_turn = turn['turn_id']
        else:
            turn_acc+=1
    
    result = []
    for l in range(len(lst_lambda)):
        fga_acc = round(fga[l]*100.0/total, 2)
        result.append(f"FGA with L={lst_lambda[l]} : {fga_acc}")
    
    return result

In [10]:
path = 'expts/230419_2058-gpt35_turbo_5p_v2_slot_classify_with_bracket_0to193'

print("\n".join(FGA(path)))

FGA with L=0.25 : 50.61
FGA with L=0.5 : 58.27
FGA with L=0.75 : 62.94
FGA with L=1.0 : 65.99


In [6]:
import json

def our_pred_parse_with_bracket(pred):
    if pred == "":
       return {}
    
    pred_slot_values = {}

    value_assigner = "="
    slot_value = pred.split(',')
    for i in slot_value:
        i = i.replace("(","").replace(")","")
        if value_assigner not in i:
           continue
        else:
           pred_slot_values[i.split(value_assigner)[0].strip()] = i.split(value_assigner)[1].strip()

    return pred_slot_values

def check_prefix_suffix(value, candidates):
    # add/delete "the" in the front, or the suffix in the end.
    if value in candidates:
        return value
    prefixes = ['the ']
    suffixes = [" hotel", " restaurant", ' cinema', ' guest house',
                " theatre", " airport", " street", ' gallery', ' museum']
    for prefix in prefixes:
        if value.startswith(prefix):
            value = value[len(prefix):]
            break
    for suffix in suffixes:
        if value.endswith(suffix):
            value = value[:-len(suffix)]
            break
    for prefix in [''] + prefixes:
        for suffix in [''] + suffixes:
            possible_value = prefix + value + suffix
            if possible_value in candidates:
                return possible_value
    return ''

def typo_fix(slot_values, ontology, version="2.1"):

    # fix the named entities in these slots
    named_entity_slots = ['hotel-name', 'train-destination', 'train-departure',
                          'attraction-type', 'attraction-name',
                          'restaurant-name', 'taxi-departure', 'taxi-destination', 'restaurant-food']
    fixed = {}
    for slot, value in slot_values.items():
        # lower
        value = value.lower()

        # fix 's
        value = value.replace(' s ', 's ')
        if value.endswith(' s'):
            value = value[:-2] + 's'
        
        # value에 ' 있으면 제거
        if "'" in value:
            value = value.replace("'","")

        # fix typo words
        general_typos = {'fen ditton': 'fenditton',
                         'guesthouse': 'guest house',
                         'steveage': 'stevenage',
                         'stantsted': 'stansted',
                         'storthford': 'stortford',
                         'shortford': 'stortford',
                         'weish': 'welsh',
                         'bringham': 'birmingham',
                         'liverpoool': 'liverpool',
                         'petersborough': 'peterborough',
                         'el shaddai': 'el shaddia',
                         'wendesday': 'wednesday',
                         'brazliian': 'brazilian',
                         'graffton': 'grafton'}
        for k, v in general_typos.items():
            value = value.replace(k, v)

        # fix whole value
        value_replacement = {'center': 'centre',
                             'caffe uno': 'cafe uno',
                             'caffee uno': 'cafe uno',
                             'christs college': 'christ college',
                             'churchill college': 'churchills college',
                             'sat': 'saturday',
                             'saint johns chop shop house': 'saint johns chop house',
                             'good luck chinese food takeaway': 'good luck',
                             'asian': 'asian oriental',
                             'gallery at 12': 'gallery at 12 a high street'}

        if version == "2.1":
            value_replacement['portuguese'] = 'portugese'
            value_replacement['museum of archaeology and anthropology'] = 'museum of archaelogy and anthropology'

        if version == "2.4":
            value_replacement['portugese'] = 'portuguese'
            value_replacement['museum of archaelogy and anthropology'] = 'museum of archaeology and anthropology'

        for k, v in value_replacement.items():
            if value == k:
                value = v

        # time format fix  9:00 -> 09:00
        if ':' in value and len(value) < 5:
            value = '0' + value

        if slot in named_entity_slots:
            value = check_prefix_suffix(value, ontology[slot])

        if value:
            fixed[slot] = value
    return fixed

with open('./data/mwz2.4/ontology.json','r') as f:
    ontology = json.load(f)
    
p = "(attraction-name = all saints church) and (attraction-location = centre)."
p = our_pred_parse_with_bracket(p)
print(p)
typo_fix(p,ontology,2.4)

{'attraction-name': 'all saints church and attraction-location'}


{}

In [7]:
slot_description = {
    "hotel-pricerange":["price budget of the hotel", "preferred cost of the hotel"],
    "hotel-type": ["what is the type of the hotel", "type of hotel building"],
    "hotel-parking":["parking facility at the hotel", "whether the hotel has parking", "does the hotel have parking"],
    "hotel-book stay":["length of stay at the hotel", "how many days you want to stay at the hotel for"],
    "hotel-book day":["day of the hotel booking", "what day of the week you want to start staying at the hotel"],
    "hotel-book people":["number of people for the hotel booking", "how many people are staying at the hotel"],
    "hotel-area":["area or place of the hotel", "rough location of the hotel", "preferred location of the hotel"],
    "hotel-stars":["star rating of the hotel", "rating of the hotel out of five stars"],
    "hotel-internet":["internet option at the hotel", "whether the hotel has internet"],
    "hotel-name":["name of the hotel", "which hotel are you looking for"],
    "train-destination":["destination of the train", "what train station you want to travel to", "destination or drop-off location of the train"],
    "train-day":["day of the train", "what day you want to take the train"],
    "train-departure":["departure location of the train", "what train station you want to leave from"],
    "train-arriveby":["arrival time of the train", "what time you want the train to arrive at your destination station by", "when you want to arrive at your destination by train"],
    "train-book people":["number of people booking for train", "how many people you need train booking for", "how many train tickets you need"],
    "train-leaveat":["leaving time for the train", "what time you want the train to leave your departure station by", "when you want to arrive at your destination by train"],
    "attraction-type":["type of the attraction", "type of attraction or point of interest"],
    "attraction-area":["area or place of the attraction", "area to search for attractions", "preferred location for attraction"],
    "attraction-name":["name of the attraction", "which attraction are you looking for"],
    "restaurant-book people":["number of people booking the restaurant", "how many people for the restaurant reservation"],
    "restaurant-book day":["day of the restaurant booking", "what day of the week to book the table at the restaurant"],
    "restaurant-book time":["time of the restaurant booking", "what time to book the table at the restaurant" ],
    "restaurant-food":["food type for the restaurant", "the cuisine of the restaurant you are looking for"],
    "restaurant-pricerange":["price budget for the restaurant", "preferred cost of the restaurant"],
    "restaurant-name":["name of the restaurant", "which restaurant are you looking for"],
    "restaurant-area":["area or place of the restaurant", "preferred location of restaurant"],
    "taxi-leaveat":["leaving time of taxi", "when you want the taxi to pick you up", "what time you want the taxi to leave your departure location by"],
    "taxi-destination":["destination of taxi", "where you want the taxi to drop you off", "what place do you want the taxi to take you to"],
    "taxi-departure":["departure location of taxi", "where you want the taxi to pick you up", "what place do you want to meet the taxi"],
    "taxi-arriveby":["arrival time of taxi", "when you want the taxi to drop you off at your destination", "what time you to arrive at your destination by taxi"]}

slot_classify_prompt = """Your task is to find the changed domain-slots based on the context and the dialogue between user and system, and find the corresponding value.
The following lists are domain-slots and their possible values.
All domain-slots have their own number that represent themselves in examples.

domain-slots (number): possible values
hotel-name (0): a and b guest house, ashley hotel, el shaddia guest house, etc.
hotel-pricerange (1): dontcare, cheap, moderate, expensive
hotel-type (2): hotel, guest house
hotel-parking (3): dontcare, yes, no
hotel-book stay (4): 1, 2, 3, etc.
hotel-book day (5): monday, tuesday, etc.
hotel-book people (6): 1, 2, 3, etc.
hotel-area (7): dontcare, centre, east, north, south, west
hotel-stars (8): dontcare, 0, 1, 2, 3, 4, 5
hotel-internet (9): dontcare, yes, no
train-destination (10): london kings cross, cambridge, peterborough, etc.
train-departure (11): cambridge, stansted airport, etc.
train-day (12): monday, saturday, etc.
train-book people (13): 1, 2, 3, etc.
train-leaveat (14): 20:24, 12:06, etc.
train-arriveby (15): 05:51, 20:52, etc.
attraction-name (16): abbey pool and astroturf pitch, adc theatre, all saints church, castle galleries, etc.
attraction-area (17): dontcare, centre, east, north, south, west
attraction-type (18): architecture, boat, church, cinema, college, concert hall, entertainment, hotspot, multiple sports, museum, nightclub, park, special, swimming pool, theatre
restaurant-name (19): pizza hut city centre, the missing sock, golden wok, cambridge chop house, darrys cookhouse and wine shop, etc.
restaurant-food (20): italian, international, chinese, dontcare, modern european, etc.
restaurant-pricerange (21): dontcare, cheap, moderate, expensive
restaurant-area (22): centre, east, north, south, west
restaurant-book time (23): 13:30, 17:11, etc.
restaurant-book day (24): wednesday, friday, etc.
restaurant-book people (25): 1, 2, 3, etc.
taxi-destination (26): copper kettle, magdalene college, lovell lodge
taxi-departure (27): royal spice, university arms hotel, da vinci pizzeria
taxi-leaveat (28): 14:45, 11:15, etc.
taxi-arriveby (29): 15:30, 12:45, etc.
"""

slot_idx_dict = {'hotel-name': 0,
 'hotel-pricerange': 1,
 'hotel-type': 2,
 'hotel-parking': 3,
 'hotel-book stay': 4,
 'hotel-book day': 5,
 'hotel-book people': 6,
 'hotel-area': 7,
 'hotel-stars': 8,
 'hotel-internet': 9,
 'train-destination': 10,
 'train-departure': 11,
 'train-day': 12,
 'train-book people': 13,
 'train-leaveat': 14,
 'train-arriveby': 15,
 'attraction-name': 16,
 'attraction-area': 17,
 'attraction-type': 18,
 'restaurant-name': 19,
 'restaurant-food': 20,
 'restaurant-pricerange': 21,
 'restaurant-area': 22,
 'restaurant-book time': 23,
 'restaurant-book day': 24,
 'restaurant-book people': 25,
 'taxi-destination': 26,
 'taxi-departure': 27,
 'taxi-leaveat': 28,
 'taxi-arriveby': 29}

for idx, k in enumerate(slot_idx_dict.keys()):
#     slot_classify_prompt = slot_classify_prompt.replace(k,"'" + k + "'")
    slot_classify_prompt = slot_classify_prompt.replace(" (" + str(idx) + ")","")
    slot_classify_prompt = slot_classify_prompt.replace(k,"(" + str(idx) + ") " + k + " (" + slot_description[k][0] + ")")

# print(slot_classify_prompt)